In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("project.ipynb")

# Project 3: Language Models 🗣

## DSC 80, Spring 2025

### Checkpoint Due Date: Friday, May 9th at 11:59PM (Questions 1-4)
### Due Date: Friday, May 16th at 11:59PM

## Instructions

---

### Working on the Project

This Jupyter Notebook contains the statements of the problems and provides code and Markdown cells to display your answers to the problems.

* Like the lab, your coding work will be developed in the accompanying `project.py` file, that will be imported into the current notebook. This code will be autograded.
* **For the Checkpoint, which is required, you only need to turn in a `project.py` containing solutions for Questions 1-4!**
    - The "Project 3 Checkpoint" autograder on Gradescope does not thoroughly check your code – it only runs the public tests on Questions 1-4 to make sure that you have completed them. There are no hidden tests for the checkpoint, and you will see your score upon submission. 
    - When you submit the final version of the project, however, we will use hidden tests to check your answers more thoroughly.
    - Note that this means you will ultimately have to submit the project twice – once to the "Project 3 Checkpoint" autograder (Questions 1-4 only), and once to the "Project 3" autograder (once you're fully done).
- **Do not change the function names in `project.py` file!** The functions in `project.py` are how your assignment is graded, and they are graded by their name. If you changed something you weren't supposed to, you can find the original code in the [course GitHub repository](https://github.com/dsc-courses/dsc80-2025-2i).
- **To ensure that all of your work to be submitted is in `project.py`, we've included a script named `project-validation.py` in the project folder. You shouldn't edit it, but instead, you should call it from the command line (e.g. the Terminal) to test your work.** More details on its usage are given at the bottom of this notebook.
- You are encouraged to write your own additional helper functions to solve the project, as long as they also end up in `project.py`.

### Warning! 🚨

Many questions in the project intentionally build off of each other and the final result matters! In fact, you can "get a question correct," but only receive partial credit for it because a previous answer was wrong.

### Working with a Partner 👯

You may work together on projects (and projects only!) with a partner. If you work with a partner, you are both required to actively contribute to all parts of the project. You must both be working on the assignment at the same time together, either physically or virtually on a Zoom call. You are encouraged to follow the pair programming model, in which you work on just a single computer and alternate who writes the code and who thinks about the problems at a high level.

In particular, you **cannot** split up the project and each work on separate parts independently.

Note that if you do work with a partner, you and your partner must submit the Checkpoint together and the whole project together. See [here](https://dsc80.com/syllabus/#projects) for more details.


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
import requests
import time

In [4]:
from project import *

d:\UCSD\DSC80\dsc80-2025-sp\projects\project03\project.py:65: SyntaxWarning: invalid escape sequence '\w'
  token_pattern = re.compile(r"[\w'-]+|[^\w\s]")


## About the Assignment

### Introduction to Language Models (LMs)

You've almost certainly heard about ChatGPT, a large language model developed by OpenAI. It is designed to generate human-like responses to natural language input.

<center><img src="imgs/chatgpt.png" width=60%>Inception!</center>

In this project, you will build your own [language models](https://en.wikipedia.org/wiki/Language_model), using publicly-available books found on [Project Gutenberg](https://www.gutenberg.org). Language models attempt to capture the likelihood that a given sequence of words occur in a given collection of texts, known as a "corpus." For instance, a language model can assign probabilities to the sentences `'when I drink Coke I smile'` and `'when I drink pizza I smile'`. One would expect the former sentence to have a higher probability than the former 😊.

As with all statistical models, the true data generating process is unknown to us, so all we can do is **estimate** the probabilities of sentences. For example, one might estimate the probability of a sentence as simply the product of the empirical probabilities (i.e., the number of times a word is observed in a dataset divided by the number of words in that dataset). In the above example, we may have:

$$P(\text{when I drink Coke I smile}) = P(\text{when}) \cdot P(\text{I}) \cdot P(\text{drink}) \cdot P(\text{Coke}) \cdot P(\text{I}) \cdot P(\text{smile})$$

We'll look at several estimation methods in this project (including the one above); as you'll see, some are more sophisticated than others. Don't worry – we'll re-introduce all of the relevant mathematics as it becomes necessary throughout the project.

### Tokenizing a Corpus

To train a language model on a corpus, we must break the corpus into a sequence of words. In reality though, the sequences are not made up entirely of words, but rather more general terms called *tokens*, which may include punctuation and other terms. The process of separating a corpus into its tokens is called **tokenization**.

Below are a few examples of tokens that aren't "words":

* Punctuation. For example, the period makes sense as a token, as certain words tend to end sentences (i.e. appear right before a `'.'`), while other words tend to begin sentences (i.e. appear right after a `'.'`).
* We have special "START" and "END" tokens that begin and end every word sequence (in our case, paragraphs of words in a given book). These make sense as tokens, as certain words may tend to begin and end paragraphs. 

It is useful for the tokens used to represent START and END to be single characters that can never be found in the text of the book you use to create your language model. Thus, you will use two ASCII hidden "[control characters](https://en.wikipedia.org/wiki/C0_and_C1_control_codes#C0_(ASCII_and_derivatives))":
* For START, you will use the character `'\x02'`, which refers to the "beginning of text".
* For END, you will use the character `'\x03'`, which refers to the "end of text".

### Checking Your Work for Correctness

To build a language model, you will need to perform several steps, from data cleaning to implementing mathematical formulas. **Things will get messy, and it can be difficult to assess the correctness of your work.**

- The `grader.check` public notebook tests will be informative, but as per usual, they won't guarantee that your code is correct.
- Run your functions on **real books** taken from [Project Gutenberg](https://www.gutenberg.org/) and make sure that their behavior looks correct. After selecting a book ([example](https://www.gutenberg.org/ebooks/68085/)), click the "Plain Text UTF-8" link to find the book's text ([example](https://www.gutenberg.org/cache/epub/68085/pg68085.txt)).

---

### Navigating the Project

Below is an outline of the the project. If there are terms you don't understand in following descriptions, you should review the introduction for an explanation of the terms and ideas described.

- [✅Part 1: Dissecting the Corpus 🪚🫀](#part1)
    - [Question 1: Preparing the Corpus](#question1)
    - [Question 2: Tokenizing the Corpus](#question2)
- [✅Part 2: Creating Baseline Language Models 📕](#part2)
    - [Question 3: Uniform Language Models](#question3)
    - [Question 4: Unigram Language Models](#question4)
- [Part 3: Creating an N-Gram Language Model 📚](#part3)
    - [Question 5.1: Creating N-Grams](#question5a)
    - [Question 5.2: Training the N-Gram LM](#question5b)
    - [Question 5.3: Computing Probabilities using the N-Gram Model](#question5c)
    - [Question 5.4: Sampling from the N-Gram Model](#question5d)
    
**Disclaimer: You should expect Question 5/Part 3 to take the same amount of time as Parts 1 and 2 combined.** Do not leave it to the last minute just because it looks like there is only "one question"! Note also that the entire project is worth **100 points**, of which **50 come from Question 5/Part 3 alone**.

Good luck – let's get started! 🎉
    
---

While working on the project, check the Ed post titled "Project 3 Released!" for any clarifications.

---

<!-- - [Part 1: Dissecting the Corpus 🪚🫀](#part1)
    - [Question 1: Preparing the Corpus (Checkpoint Question)](#question1)
    - [Question 2: Tokenizing the Corpus (Checkpoint Question)](#question2)
- [Part 2: Creating Baseline Language Models 📕](#part2)
    - [Question 3: Uniform Language Models (Checkpoint Question)](#question3)
    - [Question 4: Unigram Language Models (Checkpoint Question)](#question4) -->

## Part 1: Dissecting the Corpus 🪚🫀

<a name='part1'></a>

### Question 1 – Preparing the Corpus

<a name='question1'></a>

In this question, you will use the `requests` module to download the "Plain Text UTF-8" text of a public domain book from [Project Gutenberg](https://www.gutenberg.org/) and prepare it for analysis in later questions. For instance, the book Beowulf's "Plain Text UTF-8" URL is [here](https://www.gutenberg.org/ebooks/16328.txt.utf-8), which can be accessed by clicking the "Plain Text UTF-8" link [here](https://www.gutenberg.org/ebooks/16328).

Complete the implementation of the function `get_book`, which takes in the `url` of a "Plain Text UTF-8" book and **returns a string** containing the contents of the book. 

The returned string should satisfy the following conditions:
* The contents of the book consist of **everything** between Project Gutenberg's START and END comments (but not including the START and END comments themselves).
* The contents **do** include the title, author, and table of contents.
* You should replace any Windows newline characters (`'\r\n'`) with standard newline characters (`'\n'`).
* You should check Project Gutenberg's [`robots.txt`](https://gutenberg.org/robots.txt) as well and implement a "pause" in your request in accordance with the website's policy. If the function is called twice in succession, it should not violate the `robots.txt` policy.
    - If the `robots.txt` website does not specify a pause length, use a default pause of 0.5 seconds.

***Note***:
- There is no need to use BeautifulSoup here, so please don't import it.
- You are encouraged to find whatever books on the website that interest you to test your code and the language models you develop. The text doesn't even need to be an English-language book.
- In the public test example, the START comment is `*** START OF THE PROJECT GUTENBERG EBOOK BEOWULF: AN ANGLO-SAXON EPIC POEM ***`. The END comment is formatted similarly. Browse through several book text files before writing your function.

In [5]:
raw = requests.get('https://www.gutenberg.org/cache/epub/16328/pg16328.txt')
text = raw.text
time.sleep(0.5)

title_start = text.find('Title: ')
title_finding = text[title_start + 7:]
title_end = title_finding.find('\r')
title = title_finding[:title_end]

start_token = f'*** START OF THE PROJECT GUTENBERG EBOOK {title.upper()} ***'
end_token = f'*** END OF THE PROJECT GUTENBERG EBOOK {title.upper()} ***'
start_index = text.find(start_token)
end_index = text.find(end_token)

book_content = text[start_index: end_index + len(end_token)]

book_content = book_content.replace('\r\n', '\n')

In [6]:
beowulf = get_book('https://www.gutenberg.org/ebooks/16328.txt.utf-8')
print(beowulf)

laurel = get_book('https://www.gutenberg.org/cache/epub/76048/pg76048.txt')
print(laurel)






BEOWULF
AN ANGLO-SAXON EPIC POEM

_TRANSLATED FROM THE HEYNE-SOCIN TEXT_

BY

JNO: LESSLIE HALL, Ph. D. (J.H.U.)

Professor of English and History in The College of William and Mary


D.C. HEATH & CO., PUBLISHERS
BOSTON NEW YORK CHICAGO


Entered according to Act of Congress, in the year 1892, by

JNO: LESSLIE HALL,

in the Office of the Librarian of Congress, at Washington.


TO

My Wife

[v]




CONTENTS.


                                                               PAGE

      Preface                                                   vii

      Bibliography of Translations                               xi

      Glossary of Proper Names                                 xiii

      List of Words and Phrases not in General Use            xviii

      The Life and Death of Scyld (I.)                            1

      Scyld's Successors
                                 } (II.)                          3
      Hrothgar's Great Mead-Hall

      Grendel, the Murderer (III.)       

In [7]:
beowulf[:20]

'\n\n\n\n\nBEOWULF\nAN ANGL'

In [8]:
# don't change this cell, but do run it -- it is needed for the tests
beowulf = get_book('https://www.gutenberg.org/ebooks/16328.txt.utf-8')

In [9]:
grader.check("q1")

q1 results: All test cases passed!

### Question 2 – Tokenizing the Corpus

<a name='question2'></a>

Now, you need to **tokenize** the text of a book. To do so, complete the implementation of the function `tokenize`, which takes in a string, `book_string`, and returns a **list of the tokens** (words, numbers, and all punctuation) in the book such that:
* The start of every paragraph is represented in the list with the single character `'\x02'` (standing for START).
* The end of every paragraph is represented in the list with the single character `'\x03'` (standing for STOP).
* Tokens include *no* whitespace.
* Two or more newlines count as a paragraph break, and whitespace (e.g. multiple newlines) between two paragraphs of text do not appear as tokens.
* All punctuation marks except `_` count as tokens, even if they are uncommon (e.g. `'@'`, `'+'`, and `'%'` are all valid tokens).

For example, consider the following excerpt. (The first sentence is at the end of a larger paragraph, and the second sentence is at the start of a longer paragraph.)
```
...
My phone's dead.

I didn't get_your call!!
...
```
should tokenize to:
```py
[...
'My', 'phone', "'", 's', 'dead', '.', '\x03', '\x02', 'I', 'didn', "'", 't', 'get_your', 'call', '!', '!'
...]
```

Let's take another example:

    tokenize("\n\n\n\nBEOWULF\n\n\nAN\nANGLO")

Should produce:

```py
['\x02', 'BEOWULF', '\x03', '\x02', 'AN', 'ANGLO', '\x03']
```



***Note***:
- The the list of tokens you return should always start with `'\x02'` and end with `'\x03'` (to signify the start and end of the text) regardless of what the text input is.
- `tokenize` should run quickly; you should avoid loops. Specifically, `tokenize` should run on the the complete works of Shakespeare (in `data/shakespeare.txt`) in **under 5 seconds** to get full credit.

In [10]:
book_string = open(Path('data') / 'shakespeare.txt').read()

paragraphs = re.split(r'(?:\n){2,}', book_string.strip())

tokens = ['\x02']
    
# Tokenizing pattern: match words (including apostrophes), numbers, and punctuation
token_pattern = re.compile(r"[\w'-]+|[^\w\s]")
    
for paragraph in paragraphs:
        # Tokenize each paragraph using the regex pattern
    paragraph_tokens = token_pattern.findall(paragraph)
        
        # Add the paragraph tokens to the overall list
    tokens.extend(paragraph_tokens)
        
        # After the paragraph tokens, add the STOP token
    tokens.append('\x03')
        
        # Add the START token for the next paragraph
    tokens.append('\x02')

In [11]:
paragraphs = re.split(r'(?:\n){2,}', beowulf.strip())
beowulf

'\n\n\n\n\nBEOWULF\nAN ANGLO-SAXON EPIC POEM\n\n_TRANSLATED FROM THE HEYNE-SOCIN TEXT_\n\nBY\n\nJNO: LESSLIE HALL, Ph. D. (J.H.U.)\n\nProfessor of English and History in The College of William and Mary\n\n\nD.C. HEATH & CO., PUBLISHERS\nBOSTON NEW YORK CHICAGO\n\n\nEntered according to Act of Congress, in the year 1892, by\n\nJNO: LESSLIE HALL,\n\nin the Office of the Librarian of Congress, at Washington.\n\n\nTO\n\nMy Wife\n\n[v]\n\n\n\n\nCONTENTS.\n\n\n                                                               PAGE\n\n      Preface                                                   vii\n\n      Bibliography of Translations                               xi\n\n      Glossary of Proper Names                                 xiii\n\n      List of Words and Phrases not in General Use            xviii\n\n      The Life and Death of Scyld (I.)                            1\n\n      Scyld\'s Successors\n                                 } (II.)                          3\n      Hrothgar\'s G

In [12]:
paragraphs

['BEOWULF\nAN ANGLO-SAXON EPIC POEM',
 '_TRANSLATED FROM THE HEYNE-SOCIN TEXT_',
 'BY',
 'JNO: LESSLIE HALL, Ph. D. (J.H.U.)',
 'Professor of English and History in The College of William and Mary',
 'D.C. HEATH & CO., PUBLISHERS\nBOSTON NEW YORK CHICAGO',
 'Entered according to Act of Congress, in the year 1892, by',
 'JNO: LESSLIE HALL,',
 'in the Office of the Librarian of Congress, at Washington.',
 'TO',
 'My Wife',
 '[v]',
 'CONTENTS.',
 '                                                               PAGE',
 '      Preface                                                   vii',
 '      Bibliography of Translations                               xi',
 '      Glossary of Proper Names                                 xiii',
 '      List of Words and Phrases not in General Use            xviii',
 '      The Life and Death of Scyld (I.)                            1',
 "      Scyld's Successors\n                                 } (II.)                          3\n      Hrothgar's Great M

In [13]:
shakes = tokenize(open(Path('data') / 'shakespeare.txt').read())
shakes

['\x02',
 'The',
 'Complete',
 'Works',
 'of',
 'William',
 'Shakespeare',
 '\x03',
 '\x02',
 'by',
 'William',
 'Shakespeare',
 '\x03',
 '\x02',
 'Contents',
 '\x03',
 '\x02',
 'THE',
 'SONNETS',
 '\x03',
 '\x02',
 'ALL',
 '’',
 'S',
 'WELL',
 'THAT',
 'ENDS',
 'WELL',
 '\x03',
 '\x02',
 'THE',
 'TRAGEDY',
 'OF',
 'ANTONY',
 'AND',
 'CLEOPATRA',
 '\x03',
 '\x02',
 'AS',
 'YOU',
 'LIKE',
 'IT',
 '\x03',
 '\x02',
 'THE',
 'COMEDY',
 'OF',
 'ERRORS',
 '\x03',
 '\x02',
 'THE',
 'TRAGEDY',
 'OF',
 'CORIOLANUS',
 '\x03',
 '\x02',
 'CYMBELINE',
 '\x03',
 '\x02',
 'THE',
 'TRAGEDY',
 'OF',
 'HAMLET',
 ',',
 'PRINCE',
 'OF',
 'DENMARK',
 '\x03',
 '\x02',
 'THE',
 'FIRST',
 'PART',
 'OF',
 'KING',
 'HENRY',
 'THE',
 'FOURTH',
 '\x03',
 '\x02',
 'THE',
 'SECOND',
 'PART',
 'OF',
 'KING',
 'HENRY',
 'THE',
 'FOURTH',
 '\x03',
 '\x02',
 'THE',
 'LIFE',
 'OF',
 'KING',
 'HENRY',
 'THE',
 'FIFTH',
 '\x03',
 '\x02',
 'THE',
 'FIRST',
 'PART',
 'OF',
 'HENRY',
 'THE',
 'SIXTH',
 '\x03',
 '\x02',
 'THE

Run the cell below to prepare the autograder tests. It runs your `tokenize` function on the full body of Shakespeare's work. As a guide, you should expect the first three elements of `shakes` to be `'\x02'`, `'The'`, and `'Complete'`, and the last three elements of `shakes` to be `'William'`, `'Shakespeare'`, and `'\x03'`.

***Note***: If you are running into issues with opening the `'data/shakespeare.txt'` file, add the argument `encoding='UTF-8'` to the `open()` function.

In [16]:
# don't change this cell, but do run it -- it is needed for the tests
import time
start = time.time()
shakes = tokenize(open(Path('data') / 'shakespeare.txt').read())
elapsed = time.time() - start
elapsed # Should be (much) under 5

2.818429946899414

In [17]:
grader.check("q2")

q2 results: All test cases passed!

---

<a name='part2'></a>

## Part 2: Creating Baseline Language Models 📕

Now that we're able to tokenize a corpus, it is time to start building language models (LM).

In this project, we will build three different language models. They all operate under the premise of assigning probabilities to sentences. Given a sentence – that is, a sequence of tokens $w = w_1\ldots w_n$ – we want to be able to compute the **probability** that sentence is used: 
$$P(w) = P(w_1,\ldots,w_n)$$

However, sentences are built from tokens, and the likelihood that a token occurs where it does depends on the tokens before it. This points to using **conditional probability** to compute $P(w)$. That is, we can write:

$$
P(w) = P(w_1,\ldots,w_n) = P(w_1) \cdot P(w_2|w_1) \cdot P(w_3|w_1,w_2) \cdot\ldots\cdot P(w_n|w_1,\ldots,w_{n-1})
$$  
This is also called the **chain rule** for probabilities.

**Example**: Consider the sentence 

<center><code>'when I drink Coke I smile'</code></center>
    
The probability that it occurs, according the the chain rule, is

$$
P(\text{when}) \cdot P(\text{I | when}) \cdot P(\text{drink | when I})\cdot P(\text{Coke | when I drink}) \cdot P(\text{I | when I drink Coke}) \cdot P(\text{smile | when I drink Coke I})
$$

That is, the probability that the sentence occurs is the product of the probability that each subsequent token follows the tokens that came before. For example, the probability $P(\text{Coke | when I drink})$ is likely pretty high, as Coke is something that you drink. The probability $P(\text{pizza | when I drink})$ is likely low, because pizza is not something that you drink.

You may wonder how the language model "knows" that Coke is something that you drink, but pizza is not. The way that the language model "learns" its probabilities is by **looking at examples of existing sentences**, i.e. by being **trained on a corpus**. Throughout Parts 2 and 3, you will look at **different ways of estimating these probabilities**. In each case, you will use a corpus to assign probabilities to different tokens and combinations of tokens, and use those probabilities to generate new sentences.

<br>

Each language model you build will be a **class** with a few methods in common:

* The `__init__` constructor: when you instantiate an LM object, you will need to pass the "training corpus" on which your model will be trained (i.e. a list of tokens you created in Question 2 with `tokenize`). The `train` method will then use that data to create a model which is saved in the `mdl` attribute. This code is given to you.
* The `train` method takes in a list of tokens (e.g. the output of `tokenize`) and outputs a language model. **This language model is represented as a `Series`, whose index consists of tokens and whose values are the probabilities that the tokens occur.** (In the case of the N-Gram model in Part 3/Question 5, the model will be represented as a DataFrame instead of a Series – more on this later.)
* The `probability` method takes in a **non-empty** sequence of tokens and returns the probability that this sequence occurs under the language model.
* The `sample` method takes in a positive integer `M` and generates a string made up of `M` tokens using the language model. **This method generates random sentences!**

The description of Question 3 walks through in detail how each of these methods work. However, here's the general workflow:

$$\text{initialize LM with tokenized corpus} \rightarrow \text{train LM (i.e. assign probabilities to each token) using corpus} \rightarrow \text{used trained LM to compute probabilities of input sentences and generate random sentences}$$

In Questions 3, 4, and 5, you will create classes for different language models – uniform, unigram, and N-Gram, respectively. In each class, you will implement each of the above methods.

### Question 3 – Uniform Language Models

<a name='question3'></a>

A uniform language model is one in which each **unique** token is equally likely to appear in any position, unconditional of any other information.

Let's put into context what this means by using the following example corpus:

```py
>>> corpus = 'when I eat pizza, I smile, but when I drink Coke, my stomach hurts'
>>> tokenize(corpus)
['\x02', 'when', 'I', 'eat', 'pizza', ',', 'I', 'smile', ',', 'but', 'when', 'I', 'drink', 'Coke', ',', 'my', 'stomach', 'hurts', '\x03']
```

Given a tokenized corpus, you need to build the language model itself in `train`. As mentioned above, language models are stored as Series, where words are the index and probabilities are the values. **In a uniform language model**, the probability assigned to each token is **1 over the total number of unique tokens in the corpus**.

The example corpus above has 14 **unique** tokens. This means that we'd have $P(\text{\x02}) = \frac{1}{14}$, $P(\text{when}) = \frac{1}{14}$, and so on. Specifically, in this example, **the Series that `train` returns should contain the following values**:

| Token | Probability |
| --- | --- |
| `'\x02'` | $\frac{1}{14}$ |
| `'when'` | $\frac{1}{14}$ |
| `'I'` | $\frac{1}{14}$ |
| `'eat'` | $\frac{1}{14}$ |
| `'pizza'` | $\frac{1}{14}$ |
| `','` | $\frac{1}{14}$ |
| `'smile'` | $\frac{1}{14}$ |
| `'but'` | $\frac{1}{14}$ |
| `'drink'` | $\frac{1}{14}$ |
| `'Coke'` | $\frac{1}{14}$ |
| `'my'` | $\frac{1}{14}$ |
| `'stomach'` | $\frac{1}{14}$ |
| `'hurts'` | $\frac{1}{14}$ |
| `'\x03'` | $\frac{1}{14}$ |

***Note***:
- None of the probabilities we computed are conditional – the uniform model does not use conditional probabilities!
- When looking at the Series that `train` returns (i.e. when looking at the `mdl` attribute), the `'\x02'` and `'\x03'` characters will show up as blank characters in the index. This is to be expected.
- Your Series doesn't need to have the labels `'Token'` or `'Probability'` in it, like the above table does.

After training the model, you need to implement two more methods, `probability` and `sample`.

`probability` should take in any tuple of tokens and use the probabilities you computed in `train` (that are stored in the `mdl` attribute) to assign a probability to that sequence. For instance, suppose the input tuple is `('when', 'I', 'drink', 'Coke', 'I', 'smile')` (note that this tuple does not need to start with `'\x02'` or end with `'\x03'`). To compute the probability of this tuple under our language model, we would multiply the "trained" probabilities for each word individually. Here that would give us 

$$P(\text{when I drink Coke I smile}) = P(\text{when}) \cdot P(\text{I}) \cdot P(\text{drink}) \cdot P(\text{Coke}) \cdot P(\text{I}) \cdot P(\text{smile}) = \left( \frac{1}{14} \right)^6$$

Note that if the input tuple contains a token that was not in our corpus, `probability` should return 0.

Finally, `sample` should take in a positive integer, `M`, and return a sentence made up of `M` randomly sampled tokens, in which the probabilities come from `mdl`. For instance, if `M=5`, then we'd return a sentence containing 5 randomly selected tokens from the table above, such that the probability that each token is selected is $\frac{1}{14}$. The sampling is done with replacement, so we could end up with the same token multiple times. For instance, we might end up with `'but drink smile drink hurts'`. Note that this sentence doesn't make any grammatical sense (that's okay!) and that tokens are separated by spaces.

The starter code contains a class `UniformLM` which represents a uniform language model. Complete the implementation of the class.

In [18]:
corpus = 'when I eat pizza, I smile, but when I drink Coke, my stomach hurts'
x = tokenize(corpus)
# ['\x02', 'when', 'I', 'eat', 'pizza', ',', 'I', 'smile', ',', 'but', 'when', 'I', 'drink', 'Coke', ',', 'my', 'stomach', 'hurts', '\x03']

x = list(dict.fromkeys(x))

y = 1/len(x)
y = [y]*len(x)

z = pd.Series(y, index=x)
z

          0.071429
when       0.071429
I          0.071429
eat        0.071429
pizza      0.071429
,          0.071429
smile      0.071429
but        0.071429
drink      0.071429
Coke       0.071429
my         0.071429
stomach    0.071429
hurts      0.071429
          0.071429
dtype: float64

In [19]:
corpus = 'when I eat pizza, I smile, but when I drink Coke, my stomach hurts'
x = tokenize(corpus)

s = pd.Series(x).drop_duplicates(keep='first')

probability = 1 / len(s)

z = pd.Series(probability, index=s)

z

          0.071429
when       0.071429
I          0.071429
eat        0.071429
pizza      0.071429
,          0.071429
smile      0.071429
but        0.071429
drink      0.071429
Coke       0.071429
my         0.071429
stomach    0.071429
hurts      0.071429
          0.071429
dtype: float64

In [20]:
x = ('when', 'I', 'drink', 'Coke', 'I', 'smile')
a = z.loc[list(x)].prod()

sampled_tokens = ' '.join(np.random.choice(z.index, size=5, replace=True, p=z.values))
sampled_tokens

'eat I \x02 smile Coke'

In [21]:
tokens = tuple('one one two three one two four'.split())
unif = UniformLM(tokens)
unif.probability(('five',))
unif.probability(('five',)) == 0


True

In [22]:
# don't change this cell, but do run it -- it is needed for the tests
tokens = tuple('one one two three one two four'.split())
unif = UniformLM(tokens)
unif.sample(100)

'one four one four four one two one four two one three one two four four four one two three four four three four two two four three three two three one one three two two two three one two one one four three four two one one one three three one four one two three one three one one one two four three one one one one two one two two two one three two three one three one three one two three two three three three two three two three four three three one one two four four'

In [23]:
grader.check("q3")

q3 results: All test cases passed!

### Question 4 – Unigram Language Models

<a name='question4'></a>

A unigram language model is one in which the **probability assigned to a token is equal to the proportion of tokens in the corpus that are equal to said token**. That is, the probability distribution associated with a unigram language model is just the empirical distribution of tokens in the corpus. 

Let's understand how probabilities are assigned to tokens using our example corpus from before.

```py
>>> corpus = 'when I eat pizza, I smile, but when I drink Coke, my stomach hurts'
>>> tokenize(corpus)
['\x02', 'when', 'I', 'eat', 'pizza', ',', 'I', 'smile', ',', 'but', 'when', 'I', 'drink', 'Coke', ',', 'my', 'stomach', 'hurts', '\x03']
```

Here, there are 19 total tokens. 3 of them are equal to `'I'`, so $P(\text{I}) = \frac{3}{19}$. Here, the Series that `train` returns should contain the following values:

| Token | Probability |
| --- | --- |
| `'\x02'` | $\frac{1}{19}$ |
| `'when'` | $\frac{2}{19}$ |
| `'I'` | $\frac{3}{19}$ |
| `'eat'` | $\frac{1}{19}$ |
| `'pizza'` | $\frac{1}{19}$ |
| `','` | $\frac{3}{19}$ |
| `'smile'` | $\frac{1}{19}$ |
| `'but'` | $\frac{1}{19}$ |
| `'drink'` | $\frac{1}{19}$ |
| `'Coke'` | $\frac{1}{19}$ |
| `'my'` | $\frac{1}{19}$ |
| `'stomach'` | $\frac{1}{19}$ |
| `'hurts'` | $\frac{1}{19}$ |
| `'\x03'` | $\frac{1}{19}$ |

As before, the `probability` method should take in a tuple and return its probability, using the probabilities stored in `mdl`. For instance, suppose the input tuple is `('when', 'I', 'drink', 'Coke', 'I', 'smile')`. Then,

$$P(\text{when I drink Coke I smile}) = P(\text{when}) \cdot P(\text{I}) \cdot P(\text{drink}) \cdot P(\text{Coke}) \cdot P(\text{I}) \cdot P(\text{smile}) = \frac{2}{19} \cdot \frac{3}{19} \cdot \frac{1}{19} \cdot \frac{1}{19} \cdot \frac{3}{19} \cdot \frac{1}{19}$$

The `sample` method should now account for the fact that not all tokens are equally likely to be sampled. For instance, `'I'` is much more likely to appear in a randomly generated sentence created by `sample` than `'Coke'` is.

The starter code contains a class `UnigramLM` which represents a uniform language model. Complete the implementation of the class.

In [24]:
corpus = 'when I eat pizza, I smile, but when I drink Coke, my stomach hurts'
x = tokenize(corpus)

s = pd.Series(x)

count_series = s.value_counts()

frequency_series = count_series / len(x)

unique_tokens = pd.Series(x).drop_duplicates(keep='first')  # Keeps the first occurrence of each token
order_preserved_frequencies = frequency_series.loc[unique_tokens]

print(order_preserved_frequencies)

tokens = tuple('one one two three one two four'.split())
unigram = UnigramLM(tokens)
unigram.mdl

          0.052632
when       0.105263
I          0.157895
eat        0.052632
pizza      0.052632
,          0.157895
smile      0.052632
but        0.052632
drink      0.052632
Coke       0.052632
my         0.052632
stomach    0.052632
hurts      0.052632
          0.052632
Name: count, dtype: float64


one      0.428571
two      0.285714
three    0.142857
four     0.142857
Name: count, dtype: float64

In [25]:
# don't change this cell, but do run it -- it is needed for the tests
tokens = tuple('one one two three one two four'.split())
unigram = UnigramLM(tokens)
unigram.sample(100)

'two four four two two one two two two one three one one one one four one one two two two two one two three one two two two one two two two one two two four three one three two two three one two three one one four three one three one four four one one two one one two one one two two one one one one three one one two one three four four one three four two one one two one two one one one three two one two four two two two one one one'

In [26]:
grader.check("q4")

q4 results: All test cases passed!

### Conclusion: Baseline Language Models

You've now trained two baseline language models capable of generating new text from a given training text. Attempt to answer these questions for yourself before you continue.

* Which model do you think is better? Why?
* What are the ways in which both of these models are bad?

If you haven't trained your models on the `shakes` corpus, uncomment and run the cells below to do so and generate new random "sentences".

In [27]:
# Uncomment and run – should take less than 10 seconds
# shakes_uniform = UniformLM(shakes)
# shakes_unigram = UnigramLM(shakes)

In [28]:
# Uncomment and run
# shakes_uniform.sample(10)

In [29]:
# Uncomment and run
# shakes_unigram.sample(10)

---

<a name='part3'></a>

## Part 3: Creating an N-Gram Language Model 📚

### Recap

First, let's recap what we've done so far. Recall the chain rule for probability, where $w$ is a sentence made up of tokens $w_1, w_2, ..., w_n$:

$$P(w) = P(w_1,\ldots,w_n) = P(w_1) \cdot P(w_2|w_1) \cdot P(w_3|w_1,w_2) \cdot\ldots\cdot P(w_n|w_1,\ldots,w_{n-1})$$

In Questions 3 (Uniform) and 4 (Unigram), your `train` methods computed these probabilities **unconditionally**, meaning that the probability that a token appeared in a sentence did **not depend** on the other tokens around it. That is, you said $P(w_i | w_1, w_2, ..., w_{i - 1}) = P(w_i)$. In Question 3, you let $P(w_i) = \frac{1}{\text{\# unique tokens in corpus}}$, and in Question 4, you let $P(w_i) = \frac{\text{\# tokens in corpus equal to $w_i$}}{\text{\# tokens in corpus}}$. Cruciually, each probability was determined by looking at the corpus that the model was trained on.

<br>

### N-Gram Overview

Now we will build an N-Gram language model, in which the probability of a token appearing in a sentence **does depend** on the tokens that come before it. 

The chain rule above specifies that the probability that a token occurs at in a particular position in a sentence depends on **all** previous tokens in the sentence. However, it is often the case that the likelihood that a token appears in a sentence is influenced more by **nearby** tokens. (Remember, tokens are words, punctuation, or `'\x02'` / `'\x03'`).

The N-Gram language model relies on the assumption that only nearby tokens matter. Specifically, it assumes that the probability that a token occurs depends only on the previous $N-1$ tokens, rather than all previous tokens. That is:

$$P(w_n|w_1,\ldots,w_{n-1}) = P(w_n|w_{n-(N-1)},\ldots,w_{n-1})$$

In an N-Gram language model, there is a hyperparameter that we get to choose when creating the model, $N$. For any $N$, the resulting N-Gram model looks at the previous $N-1$ tokens when computing probabilities. (Note that the unigram model you built in Question 4 is really an N-Gram model with $N=1$, since it looked at 0 previous tokens when computing probabilities.)

<br>

#### Example: Trigram Model

When $N=3$, we have a "trigram" model. Such a model looks at the previous $N-1 = 2$ tokens when computing probabilities.

Consider the tuple `('when', 'I', 'drink', 'Coke', 'I', 'smile')`, corresponding to the sentence `'when I drink Coke I smile'`. Under the trigram model, the probability of this sentence is computed as follows:

$$P(\text{when I drink Coke I smile}) = P(\text{when}) \cdot P(\text{I | when}) \cdot P(\text{drink | when I}) \cdot P(\text{Coke | I drink}) \cdot P(\text{I | drink Coke}) \cdot P(\text{smile | Coke I})$$

The trigram model doesn't consider the beginning of the sentence when computing the probability that the sentence ends in `'smile'`.

<br>

#### N-Grams

Both when working with a training corpus and when implementing the `probability` method to compute the probabilities of other sentences, you will need to work with "chunks" of $N$ tokens at a time.

**Definition:** The **N-Grams of a text** are a list of tuples containing sliding windows of length $N$.

For instance, the trigrams in the sentence `'when I drink Coke I smile'` are:

```py
[('when', 'I', 'drink'), ('I', 'drink', 'Coke'), ('drink', 'Coke', 'I'), ('Coke', 'I', 'smile')]
```

<br>

#### Computing N-Gram Probabilities

Notice in our trigram model above, we computed $P(\text{when I drink Coke I smile})$ as being the product of several conditional probabilities. These conditional probabilities are the result of **training** our N-Gram model on a training corpus.

To train an N-Gram model, we must compute a conditional probability for every $N$-token sequence in the corpus. For instance, suppose again that we are training a trigram model. Then, for every 3-token sequence $w_1, w_2, w_3$, we must compute $P(w_3 | w_1, w_2)$. To do so, we use:

$$P(w_3 | w_1, w_2) = \frac{C(w_1, w_2, w_3)}{C(w_1, w_2)}$$

where $C(w_1, w_2, w_3)$ is the number of occurrences of the trigram sequence $w_1, w_2, w_3$ in the training corpus and $C(w_1, w_2)$ is the number of occurrences of the bigram sequence  $w_1, w_2$ in the training corpus. (Technical note: the probabilities that we compute using the ratios of counts are _estimates_ of the true conditional probabilities of N-Grams in the population of corpuses from which our corpus was drawn.)

In general, for any $N$, conditional probabilities are computed by dividing the counts of N-Grams by the counts of the (N-1)-Grams they follow. 

**In the description of Question 5.2 we provide a detailed example of how we might compute such probabilities.**

<br>

### The `NGramLM` Class

The `NGramLM` class contains a few extra methods and attributes beyond those of `UniformLM` and `UnigramLM`:

1. Instantiating `NGramLM` requires both a list of tokens and a positive integer `N`, specifying the N in N-grams. This parameter is stored in an attribute `N`.
    - Assume that `N` is less than or equal to `len(tokens)`.
1. The `NGramLM` class has a method `create_ngrams` that takes in a list of tokens and returns a list of N-Grams (recall from above, an N-Gram is a **tuple** of length N). This list of N-Grams is then passed to the `train` method to train the N-Gram model.
1. While the `train` method still creates a language model (in this case, an N-Gram model) and stores it in the `mdl` attribute, this model is most naturally stored as a DataFrame. This DataFrame will have three columns:
    - `'ngram'`, containing the N-Grams found in the text.
    - `'n1gram'`, containing the (N-1)-Grams upon which the N-Grams in `ngram` are built.
    - `'prob'`, containing the probabilities of each N-Gram in `ngram`.
1. The `NGramLM` class has an attribute `prev_mdl` that stores an (N-1)-Gram language model over the same corpus (which in turn will store an (N-2)-Gram language model over the same corpus, and so on). This is necessary to compute the probability that a word occurs at the start of a text. This code is included for you in the constructor.

### Question 5.1 – Creating N-Grams

<a name='question5a'></a>

Complete the implementation of the `create_ngrams` method of the `NGramLM` class, which takes in a list of tokens and returns a list of N-Grams, as a tuple. Example behavior is shown below.

```py
>>> corpus = 'when I eat pizza, I smile, but when I drink Coke, my stomach hurts'
>>> tokens = tokenize(corpus)
>>> tokens
['\x02', 'when', 'I', 'eat', 'pizza', ',', 'I', 'smile', ',', 'but', 'when', 'I', 'drink', 'Coke', ',', 'my', 'stomach', 'hurts', '\x03']
>>> pizza_model = NGramLM(3, tokens)
>>> pizza_model.create_ngrams(tokens)
[('\x02', 'when', 'I'),
 ('when', 'I', 'eat'),
 ('I', 'eat', 'pizza'),
 ('eat', 'pizza', ','),
 ('pizza', ',', 'I'),
 (',', 'I', 'smile'),
 ('I', 'smile', ','),
 ('smile', ',', 'but'),
 (',', 'but', 'when'),
 ('but', 'when', 'I'),
 ('when', 'I', 'drink'),
 ('I', 'drink', 'Coke'),
 ('drink', 'Coke', ','),
 ('Coke', ',', 'my'),
 (',', 'my', 'stomach'),
 ('my', 'stomach', 'hurts'),
 ('stomach', 'hurts', '\x03')]
```

Make sure you understand the above behavior before implementing `create_ngrams`.

In [30]:
corpus = 'when I eat pizza, I smile, but when I drink Coke, my stomach hurts'
x = tokenize(corpus)
x

z = []

window_size = 3

for i in range(len(x) - window_size + 1):
    y = tuple(x[i:i + window_size])
    z.append(y)

z



[('\x02', 'when', 'I'),
 ('when', 'I', 'eat'),
 ('I', 'eat', 'pizza'),
 ('eat', 'pizza', ','),
 ('pizza', ',', 'I'),
 (',', 'I', 'smile'),
 ('I', 'smile', ','),
 ('smile', ',', 'but'),
 (',', 'but', 'when'),
 ('but', 'when', 'I'),
 ('when', 'I', 'drink'),
 ('I', 'drink', 'Coke'),
 ('drink', 'Coke', ','),
 ('Coke', ',', 'my'),
 (',', 'my', 'stomach'),
 ('my', 'stomach', 'hurts'),
 ('stomach', 'hurts', '\x03')]

### Question 5.2 – Training the N-Gram LM

<a name='question5b'></a>

Now, you will compute the probabilities that define N-Gram language model itself. Recall that the N-Gram LM consists of probabilities of the form

$$P(w_n|w_{n-(N-1)},\ldots,w_{n-1})$$

which we estimate by  

$$\frac{C(w_{n-(N-1)}, w_{n-(N-2)}, \ldots, w_{n-1}, w_n)}{C(w_{n-(N-1)}, w_{n-(N-2)}, \ldots, w_{n-1})}$$

for every N-Gram that occurs in the corpus. To illustrate, consider again the following example corpus:

```py
>>> corpus = 'when I eat pizza, I smile, but when I drink Coke, my stomach hurts'
>>> tokens = tokenize(corpus)
>>> tokens
['\x02', 'when', 'I', 'eat', 'pizza', ',', 'I', 'smile', ',', 'but', 'when', 'I', 'drink', 'Coke', ',', 'my', 'stomach', 'hurts', '\x03']
>>> pizza_model = NGrams(3, tokens)
```

Here, `pizza_model.train` must compute $P(\text{I | \x02 when})$, $P(\text{eat | when I})$, $P(\text{pizza | I eat})$, and so on, until $P(\text{\x03 | stomach hurts})$.

To compute $P(\text{eat | when I})$, we must find the number of occurrences of `'when I eat'` in the training corpus, and divide it by the number of occurrences of `'when I'` in the training corpus. `'when I eat'` occurred exactly once in the training corpus, while `'when I'` occurred twice, so,

$$P(\text{eat | when I}) = \frac{C(\text{when I eat})}{C(\text{when I})} = \frac{1}{2}$$

To store the conditional probabilities of all N-Grams, we will use a DataFrame with three columns, like so:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>ngram</th>
      <th>n1gram</th>
      <th>prob</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>(when, I, drink)</td>
      <td>(when, I)</td>
      <td>0.5</td>
    </tr>
    <tr>
      <th>1</th>
      <td>(when, I, eat)</td>
      <td>(when, I)</td>
      <td>0.5</td>
    </tr>
    <tr>
      <th>2</th>
      <td>(,, but, when)</td>
      <td>(,, but)</td>
      <td>1.0</td>
    </tr>
    <tr>
      <th>3</th>
      <td>(,, I, smile)</td>
      <td>(,, I)</td>
      <td>1.0</td>
    </tr>
    <tr>
      <th>4</th>
      <td>(I, smile, ,)</td>
      <td>(I, smile)</td>
      <td>1.0</td>
    </tr>
    <tr>
      <th>5</th>
      <td>(,, my, stomach)</td>
      <td>(,, my)</td>
      <td>1.0</td>
    </tr>
    <tr>
      <th>6</th>
      <td>(but, when, I)</td>
      <td>(but, when)</td>
      <td>1.0</td>
    </tr>
    <tr>
      <th>7</th>
      <td>(, when, I)</td>
      <td>(, when)</td>
      <td>1.0</td>
    </tr>
    <tr>
      <th>8</th>
      <td>(stomach, hurts, )</td>
      <td>(stomach, hurts)</td>
      <td>1.0</td>
    </tr>
    <tr>
      <th>9</th>
      <td>(Coke, ,, my)</td>
      <td>(Coke, ,)</td>
      <td>1.0</td>
    </tr>
    <tr>
      <th>10</th>
      <td>(eat, pizza, ,)</td>
      <td>(eat, pizza)</td>
      <td>1.0</td>
    </tr>
    <tr>
      <th>11</th>
      <td>(I, drink, Coke)</td>
      <td>(I, drink)</td>
      <td>1.0</td>
    </tr>
    <tr>
      <th>12</th>
      <td>(my, stomach, hurts)</td>
      <td>(my, stomach)</td>
      <td>1.0</td>
    </tr>
    <tr>
      <th>13</th>
      <td>(pizza, ,, I)</td>
      <td>(pizza, ,)</td>
      <td>1.0</td>
    </tr>
    <tr>
      <th>14</th>
      <td>(I, eat, pizza)</td>
      <td>(I, eat)</td>
      <td>1.0</td>
    </tr>
    <tr>
      <th>15</th>
      <td>(drink, Coke, ,)</td>
      <td>(drink, Coke)</td>
      <td>1.0</td>
    </tr>
    <tr>
      <th>16</th>
      <td>(smile, ,, but)</td>
      <td>(smile, ,)</td>
      <td>1.0</td>
    </tr>
  </tbody>
</table>

Here are a couple qualities of this DataFrame that are **important** to note:
- Each row in the DataFrame represents each unique trigram in the corpus. As such your model should not include duplicate `ngram`s (though it will likely have duplicate `n1gram`s). **To be clear, the DataFrame should _not_ have any duplicate rows!**
- Each row contains the conditional probability for its corresponding trigram. For example the row at position **1** in the above table shows that the probability of the trigram `('when', 'I', 'eat')` conditioned on the bigram `('when', 'I')` is 0.5, as we computed above. 
- Note that many of the above conditional probabilities are equal to 1 because many trigrams and their corresponding bigrams each appeared only once, and $\frac{1}{1} = 1$. 
- Note that `'\x02'` and `'\x03'` appear as spaces above, such as in row **7**.


After you've understood the above example output, complete the implementation of the `train` method in `NGramLM`. Remember that your model may use any $N$, not just 3 (i.e not just trigrams)!

In [31]:
corpus = 'when I eat pizza, I smile, but when I drink Coke, my stomach hurts'
tokens = tokenize(corpus)
bigrams = NGramLM(3, tokens)
x = bigrams.ngrams
bigrams.mdl


,ngram,n1gram,prob
0,"(when, I, eat)","(when, I)",0.5
1,"(when, I, drink)","(when, I)",0.5
2,"(I, eat, pizza)","(I, eat)",1.0
3,"(, when, I)","(, when)",1.0
4,"(pizza, ,, I)","(pizza, ,)",1.0
5,"(,, I, smile)","(,, I)",1.0
6,"(I, smile, ,)","(I, smile)",1.0
7,"(eat, pizza, ,)","(eat, pizza)",1.0
8,"(smile, ,, but)","(smile, ,)",1.0
9,"(,, but, when)","(,, but)",1.0


In [32]:
short_corpus = 'zebras eat green peas \n\n cows eat green grass \n\n zebras eat green peppers'
short_tokens = tokenize(short_corpus)
grass_model = NGramLM(2, short_tokens)
grass_model.prev_mdl.mdl

x = grass_model.ngrams

In [33]:
df = pd.DataFrame({'ngram':x})
df['count_x'] = df.groupby('ngram')['ngram'].transform('count')
df['n1gram'] = df['ngram'].apply(lambda x: x[:-1])
df['count_y'] = df.groupby('n1gram')['n1gram'].transform('count')

df['prob'] = df['count_x'] / df['count_y']

df = df[['ngram', 'n1gram', 'prob']].sort_values(by='prob').set_index('ngram').reset_index()

df = df.drop_duplicates()

df

,ngram,n1gram,prob
0,"(green, peas)","(green,)",0.333333
1,"(, cows)","(,)",0.333333
2,"(green, peppers)","(green,)",0.333333
3,"(green, grass)","(green,)",0.333333
4,"(, zebras)","(,)",0.666667
6,"(eat, green)","(eat,)",1.000000
7,"(zebras, eat)","(zebras,)",1.000000
8,"(cows, eat)","(cows,)",1.000000
9,"(, )","(,)",1.000000
10,"(peas, )","(peas,)",1.000000


In [34]:
z = []

window_size = 2

for i in range(len(x) - window_size + 1):
    y = tuple(x[i:i + window_size])
    z.append(y)

z

w = []

window_size = 3

for i in range(len(x) - window_size + 1):
    y = tuple(x[i:i + window_size])
    w.append(y)

df = pd.DataFrame({'ngram':w})
df = df.groupby('ngram', as_index=False).agg(count_x=('ngram', 'count'))
df['x'] = df['ngram'].apply(lambda x: (x[0], x[1]))

df2 = pd.DataFrame({'n1gram':z})
df2 = df2.groupby('n1gram', as_index=False).agg(count_y=('n1gram', 'count'))

final = df.merge(df2, left_on='x', right_on='n1gram', how='left')

final['prob'] = final['count_x'] / final['count_y']

final[['ngram', 'n1gram', 'prob']].sort_values(by='prob').set_index('ngram').reset_index()

,ngram,n1gram,prob
0,"((zebras, eat), (eat, green), (green, peas))","((zebras, eat), (eat, green))",0.5
1,"((zebras, eat), (eat, green), (green, peppers))","((zebras, eat), (eat, green))",0.5
2,"((, cows), (cows, eat), (eat, green))","((, cows), (cows, eat))",1.0
3,"((, zebras), (zebras, eat), (eat, green))","((, zebras), (zebras, eat))",1.0
4,"((cows, eat), (eat, green), (green, grass))","((cows, eat), (eat, green))",1.0
5,"((eat, green), (green, grass), (grass, ))","((eat, green), (green, grass))",1.0
6,"((, ), (, cows), (cows, eat))","((, ), (, cows))",1.0
7,"((, ), (, zebras), (zebras, eat))","((, ), (, zebras))",1.0
8,"((eat, green), (green, peppers), (peppers, ))","((eat, green), (green, peppers))",1.0
9,"((eat, green), (green, peas), (peas, ))","((eat, green), (green, peas))",1.0


### Question 5.3 – Computing Probabilities using the N-Gram Model

<a name='question5c'></a>

After we've trained our N-Gram model – that is, after we've computed a DataFrame associating each N-Gram with a conditional probability – we need to compute probabilities for new sentences.

To illustrate how this may work, let's look at an example input tuple to `probability`. Assume our model is `pizza_model` from Question 5.2's description; we will not repeat the probability table here.

Suppose our input tuple is `('when', 'I', 'eat', 'pizza', ',', 'I', 'smile')`, corresponding to the sentence `'when I eat pizza, I smile'` (remember again that the tuples provided to `probability` don't need to include `'\x02'` or `'\x03'`). Then,

$$
\begin{align*} &P(\text{when I eat pizza, I smile}) \\ &= P(\text{when}) \cdot P(\text{I | when}) \cdot P(\text{eat | when I}) \cdot P(\text{pizza | I eat}) \cdot P(\text{, | eat pizza}) \cdot P(\text{I | pizza,})\cdot P(\text{smile | , I}) \\ &= \frac{2}{19} \cdot 1 \cdot \frac{1}{2} \cdot 1 \cdot 1 \cdot 1 \cdot 1 \\ &= \frac{1}{19} \end{align*}
$$


- To find the latter five probabilities – $P(\text{eat | when I}) , P(\text{pizza | I eat}) , P(\text{, | eat pizza}) , P(\text{I | pizza,}),$ and $P(\text{smile | , I})$, we can use the `mdl` DataFrame that the `train` method computes.
- To find $P(\text{I | when})$, we can't just look at the `mdl` DataFrame, because `('when', 'I')` is not a trigram, it is a bigram. Instead, we look at our model's `prev_mdl` attribute, which itself is another instance of `NGramLM`, corresponding to a bigram model over the same corpus. There, we can find the probability $P(\text{I | when})$.
- To find $P(\text{when})$, we can't just look at the `mdl` DataFrame, because `'when'` is not a trigram. It is not a bigram either. Instead, we need to look at `prev_mdl`'s `prev_mdl`, which is a `UnigramLM`, to find $P(\text{when})$.

Note that if the input tuple contains an N-Gram that was never seen in the training corpus, the returned probability is 0. Convince yourself why `pizza_model.probability(('when', 'I', 'drink', 'Coke', ',', 'I', 'smile'))` is 0 before proceeding.

After you've understood the above example output, complete the implementation of the `probability` method in `NGramLM`.

In [36]:
tokens = tuple('\x02 one two three one four \x03'.split())
bigrams = NGramLM(2, tokens)
out = bigrams.create_ngrams(tokens)

bigrams.mdl
a = bigrams.prev_mdl.mdl

words = tuple('one two three one four'.split())
words

# P(one) + P(two|one) + P(three|two) + P(one|three) + P(four|one)

# P(one) + P(two|one) + P(three|one, two) + P(one|two, three) + P(four|three, one)


# (one) + ((one two), (one)) + ((two three), (two)) + ((three one), (three)) + ((one four), (one))

# (one) + ((one two), (one)) + ((one two three), (one two)) + ((two three one), (two three)) + ((three one four), (three one))

('one', 'two', 'three', 'one', 'four')

In [37]:
corpus = 'when I eat pizza, I smile, but when I drink Coke, my stomach hurts'
words = tuple('when I eat pizza, I smile'.split())
tokens = tokenize(corpus)
bigrams = NGramLM(3, tokens)

words = tokenize(' '.join(words))
words = [word for word in words if word not in ('\x02', '\x03')]

result = []

for k in range(1, bigrams.N):
    # full ngram of length k:
    full_ngram = tuple(words[:k])
    # For a unigram there's no context. For k>=2, context is the first k-1 words.
    context = full_ngram[:-1] if k > 1 else None
    result.append((full_ngram, context))

for i in range(bigrams.N - 1, len(words)):
    # Form the full ngram from words[i-N+1 : i+1]
    full_ngram = tuple(words[i - bigrams.N + 1 : i + 1])
    # The preceding context is the first N-1 items of full_ngram.
    context = full_ngram[:-1]
    result.append((full_ngram, context))

result

[(('when',), None),
 (('when', 'I'), ('when',)),
 (('when', 'I', 'eat'), ('when', 'I')),
 (('I', 'eat', 'pizza'), ('I', 'eat')),
 (('eat', 'pizza', ','), ('eat', 'pizza')),
 (('pizza', ',', 'I'), ('pizza', ',')),
 ((',', 'I', 'smile'), (',', 'I'))]

In [38]:
bigrams.prev_mdl.mdl

,ngram,n1gram,prob
0,"(I, eat)","(I,)",0.333333
1,"(,, I)","(,,)",0.333333
2,"(I, smile)","(I,)",0.333333
3,"(,, but)","(,,)",0.333333
4,"(I, drink)","(I,)",0.333333
5,"(,, my)","(,,)",0.333333
6,"(eat, pizza)","(eat,)",1.000000
7,"(, when)","(,)",1.000000
8,"(smile, ,)","(smile,)",1.000000
9,"(pizza, ,)","(pizza,)",1.000000


In [39]:
result

product = 1

for i in result:
    # print(len(i[0]))
    length = len(i[0])

    if length == bigrams.N:
        df = bigrams.mdl
        # print(type(df))
    else:
        # print('HELP')
        # print(bigrams.N - length)
        df = bigrams.prev_mdl
        for j in range(bigrams.N - length - 1):
            df = df.prev_mdl
        df = df.mdl
        # print(type(df.mdl))

    # checks if 2nd element is None
    # print(i)
    if i[1] == None:
        # print('NONE')
        p = df.loc[i[0]]
        # print(p)
    else:
        p = df[(df["ngram"] == i[0]) & (df["n1gram"] == i[1])]['prob']
        if len(p) == 0:
            p = 0
        else:
            p = p.iloc[0]
        
        # print(p)
    # print('')
    product *= p

product



np.float64(0.05263157894736842)

### Question 5.4 – Sampling from the N-Gram Model

<a name='question5d'></a>

The last method you implemented in the `UniformLM` and `UnigramLM` classes was `sample`, which gave you a way of generating new sentences. 

Now, you will implement the `sample` method in the `NGramLM` class. It should take in a positive integer `M` and generate a string of M tokens using the trained language model. It should begin with a starting token `'\x02'`, then generate a subsequent `M-1` tokens from the probabilities in `self.mdl` while continuing to pick words conditioned on the most recent previous choices, lastly a `'\x03'` token should be added to the end giving a string of `M` tokens (not counting the inital `'\x02'` token).

Let's illustrate how sampling works using a small concrete example. Suppose our corpus and **trigram** model are defined below:

```py
>>> short_corpus = 'zebras eat green peas \n\n cows eat green grass \n\n zebras eat green peppers'
>>> short_tokens = tokenize(short_corpus)
>>> short_tokens
['\x02', 'zebras', 'eat', 'green', 'peas', '\x03', '\x02', 'cows', 'eat', 'green', 'grass', '\x03', '\x02', 'zebras', 'eat', 'green', 'peppers', '\x03']
>>> grass_model = NGramLM(3, short_tokens)
```

Suppose we are told to execute `grass_model.sample(5)`. Here's how we'd proceed:

0. The first character in the output is `'\x02'`, as specified above. **We won't count this starting `'\x02'` in the length of our output string**, so we still need to find 5 more tokens.
1. The next character needs to be either `'zebras'` or `'cows'`, since `('\\x02', 'zebras')` and `('\\x02', 'cows')` are the only **bigrams** in `short_tokens` that start with an `'\\x02'`. $P(\text{zebras | \\x02})$ is $\frac{2}{3}$ and $P(\text{cows | \\x02})$ is $\frac{1}{3}$, so we select either `'zebras'` or `'cows'` for our next token according to these probabilities. For the sake of example, suppose we select `'cows'`. 4 more tokens to go.
2. Now, we must look for **trigrams** that start with the bigram `('\x02', 'cows')`. There is just one, `('\x02', 'cows', 'eat')`, so our next token must be `'eat'`. 3 more tokens to go.
3. Now, we must look for **trigrams** that start with the bigram `('cows', 'eat')`. Again, there is just one, `('cows', 'eat', 'green')`, so our next token must be `'green'`. 2 more tokens to go.
4. Now, we must look for **trigrams** that start with the bigram `('eat', 'green')`. There are three options – `('eat', 'green', 'peas')`, `('eat', 'green', 'grass')`, and `('eat', 'green', 'peppers')`. Since $P(\text{peas | eat green}) = P(\text{grass | eat green}) = P(\text{peppers | eat green}) = \frac{1}{3}$, we pick either `'peas'`, `'grass'`, or `'peppers'` uniformly at random. For the sake of example, suppose we select `'peppers'`. 1 more token to go.
5. We must end the output string now with `'\x03'`, putting us at `'\x02'` plus 5 tokens, which is the number of tokens we were told to sample. Note that `'\x03'` **does** count towards the number of tokens we were asked to sample. 

Our result is `'\x02 cows eat green peppers \x03'`, this is 5 tokens excluding the initial `'\x02'`. **Note that in our training corpus we never encountered an instance of cows 🐄 eating green peppers 🫑, but we were able to generate a coherent sentence in which they did – pretty cool!**


Some additional guidance:
- Only the initial `'\x02'` token is not counted for the number of tokens generated, every subsequent `'\x02'` or `'\x03'` token you generate will count towards that total (including the mandatory final `'\x03'`).
- If you run into a situation where there are no N-Grams that match the most recent (N-1)-Gram, you should add `'\x03'` (STOP) token as the next token in your output sentence. There is a chance that your sampled sentence ends in many `'\x03'`s, and that's fine.
- Helper functions and recursion will be very helpful.
- You will not be tested for a case where `M` (the number of tokens you should generate) is less than `N` from your ngrams.

After you've understood the above example, complete the implementation of the `sample` method in `NGramLM`.

In [40]:
short_corpus = 'zebras eat green peas \n\n cows eat green grass \n\n zebras eat green peppers'
short_tokens = tokenize(short_corpus)
grass_model = NGramLM(3, short_tokens)
df = grass_model.prev_mdl.mdl
# grass_model.mdl

filtered_df = df[df['n1gram'] == ('\x02',)]
filtered_df

chosen_ngram = np.random.choice(
    filtered_df['ngram'],
    p=filtered_df['prob'] / filtered_df['prob'].sum()  # ensure probs sum to 1
)

chosen_ngram

('\x02', 'zebras')

In [41]:
grass_model.sample(5)

'\x02 zebras eat green peas \x03'

In [60]:
tokens = tuple('\x02 one two one three one two \x03'.split())
bigrams2 = NGramLM(2, tokens)
k = bigrams2.probability('two one three'.split())
print((1/4) * (1/2) * (1/3))
k

d:\UCSD\DSC80\dsc80-2025-sp\projects\project03\project.py:65: SyntaxWarning: invalid escape sequence '\w'
  token_pattern = re.compile(r"[\w'-]+|[^\w\s]")


0.041666666666666664


np.float64(0.041666666666666664)

In [70]:
# don't change this cell, but do run it -- it is needed for the tests
tokens = tuple('\x02 one two three one four \x03'.split())
bigrams = NGramLM(2, tokens)
out = bigrams.create_ngrams(tokens)

d:\UCSD\DSC80\dsc80-2025-sp\projects\project03\project.py:65: SyntaxWarning: invalid escape sequence '\w'
  token_pattern = re.compile(r"[\w'-]+|[^\w\s]")


In [75]:
# don't change this cell, but do run it -- it is needed for the tests
tokens = "\x02 Humpty Dumpty sat on a wall , Humpty Dumpty had a great fall . \x03 \x02 All the king ' s horses and all the king ' s men couldn ' t put Humpty together again . \x03".split()
tokens = tuple(tokens)
ngram = NGramLM(2, tokens)
out_5a1 = ngram.create_ngrams(tokens)
out_5b1 = ngram.mdl
out_5c1 = ngram
out_5d1 = ngram.sample(500) 

d:\UCSD\DSC80\dsc80-2025-sp\projects\project03\project.py:65: SyntaxWarning: invalid escape sequence '\w'
  token_pattern = re.compile(r"[\w'-]+|[^\w\s]")


In [78]:
ngram2 = NGramLM(3, tokens)
out_5d2 = ngram2.sample(500)
len(out_5d2)

# out_5d2

1999

In [77]:
grader.check("q5")

q5 results: All test cases passed!

Now that you've built an N-Gram language model, let's use it to actually generate sentences!

Uncomment and run the cell below to define a bigram model using the `shakes` corpus and to generate a sentence of length 50 using the model. **The cell should run in under 30 seconds.**

In [ ]:
# Uncomment and run
# shakes_bigram = NGramLM(2, shakes)
# shakes_bigram.sample(50)

The hidden tests in Part 3/Question 5 will test your `NGramLM` implementation on corpuses that are much longer than the public tests. One of the corpuses we will test your implementation on is in `data/homertokens.txt`.

In [ ]:
homer_tokens = tuple(open(Path('data') / 'homertokens.txt').read().split(' '))

As such, it's a good idea to make sure that you can instantiate an `NGramLM` object using `homer_tokens` and that all methods (`probability`, `sample`) run in under ~20 seconds.

In [ ]:
# NGramLM(5, homer_tokens)

After you're satisfied with your `NGramLM` implementation, do a little bit of reflecting. How might you improve the `NGramLM` model? One major deficit is that it assigns a probability of 0 to sentences that contain N-Grams that weren't seen in the corpus; how might you address this? 

***Hint***: You encountered a similar issue when learning Naïve Bayes in DSC 40A! 😊

You don't need to actually make any improvements to `NGramLM`, these are just points for you to think about.

## Congratulations, you've finished Project 3! 🎉

As a reminder, all of the work you want to submit needs to be in `project.py`.

To ensure that all of the work you want to submit is in `project.py`, we've included a script named `project-validation.py` in the project folder. You shouldn't edit it, but instead, you should call it from the command line (e.g. the Terminal) to test your work.

Once you've finished the project, you should open the command line and run, in the directory for this project:

```
python project-validation.py
```

**This will run all of the `grader.check` cells that you see in this notebook, but only using the code in `project.py` – that is, it doesn't look at any of the code in this notebook. If all of your `grader.check` cells pass in this notebook but not all of them pass in your command line with the above command, then you likely have code in your notebook that isn't in your `project.py`!**

You can also use `project-validation.py` to test individual questions. For instance,

```
python project-validation.py q1 q4
```

will run the `grader.check` cells for Questions 1 and 4 – again, only using the code in `project.py`.

Once `python project-validation.py` shows that you're passing all test cases, you're ready to submit your `project.py` (and only your `project.py`) to Gradescope. Once submitting to Gradescope, make sure to stick around until all test cases pass.

There is also a call to `grader.check_all()` below in _this_ notebook, but make sure to also follow the steps above.


---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()